<a href="https://colab.research.google.com/github/AkiBae4891/DS-Unit-1-Sprint-5-Data-Storytelling-Blog-Post/blob/master/Anime_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports for the project
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [0]:
dfanime = pd.read_csv('anime.csv')
dfrating = pd.read_csv('rating.csv')



In [11]:
dfanime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [7]:
dfrating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [14]:
# I saw episodes has an input 'unknown' making the column an object.
print(dfanime.shape)
dfanime.dtypes

(12294, 7)


anime_id      int64
name         object
genre        object
type         object
episodes     object
rating      float64
members       int64
dtype: object

In [12]:
print(dfrating.shape)
dfrating.dtypes

(7813737, 3)


user_id     int64
anime_id    int64
rating      int64
dtype: object